In [19]:
#Dependencies and Setup
import pandas as pd
import math
import numpy as np
#File to Load
train_data= "clean_data_rs/ptv_metro_train_station.csv" 
listing_data= "clean_data_rs/airbnbFINAL_JS.csv"

#files and store into Pandas data frame
station_df = pd.read_csv(train_data) 
station_df.head()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP
0,19967,Anstey Railway Station (Brunswick),-37.761242,144.960684,1,Upfield
1,19968,Brunswick Railway Station (Brunswick),-37.767721,144.959587,1,Upfield
2,19969,Jewell Railway Station (Brunswick),-37.774987,144.958717,1,Upfield
3,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield
4,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield


In [20]:
listing_df = pd.read_csv(listing_data, low_memory=False)
listing_df.head()

,id,scrape_id,last_scraped,name,host_id,street,city,latitude,longitude,is_location_exact
0,9835,2.020000e+13,7/12/2018,Beautiful Room & House,33057,"Bulleen, VIC, Australia",Bulleen,-37.772684,145.092133,f
1,10803,2.020000e+13,7/12/2018,Room in Cool Deco Apartment in Brunswick,38901,"Brunswick East, VIC, Australia",Brunswick East,-37.766505,144.980736,t
2,12936,2.020000e+13,7/12/2018,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,50121,"St Kilda, VIC, Australia",St Kilda,-37.859755,144.977369,t
3,15246,2.020000e+13,7/12/2018,Large private room-close to city,59786,"Thornbury, VIC, Australia",Thornbury,-37.758971,144.989228,t
4,16760,2.020000e+13,7/12/2018,Melbourne BnB near City & Sports,65090,"St Kilda East, VIC, Australia",St Kilda East,-37.864530,144.992238,t


In [21]:
station_df.dtypes

STOP_ID         int64
STOP_NAME      object
LATITUDE      float64
LONGITUDE     float64
TICKETZONE     object
ROUTEUSSP      object
dtype: object

In [22]:
listing_df.dtypes

id                     int64
scrape_id            float64
last_scraped          object
name                  object
host_id                int64
street                object
city                  object
latitude             float64
longitude            float64
is_location_exact     object
dtype: object

In [23]:
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):

    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [24]:
def find_nearest(lat, long):
    distances = station_df.apply(
        lambda row: dist(lat, long, row['LATITUDE'], row['LONGITUDE']), 
        axis=1)
    return station_df.loc[distances.idxmin(), 'STOP_NAME']

# append the station name back to the listings file
listing_df['STOP_NAME'] = listing_df.apply(lambda row: find_nearest(row['latitude'], row['longitude']), axis=1)
# To check the data frame if it has a new column of station name (for each and every listing)
listing_df.head()

,id,scrape_id,last_scraped,name,host_id,street,city,latitude,longitude,is_location_exact,STOP_NAME
0,9835,2.020000e+13,7/12/2018,Beautiful Room & House,33057,"Bulleen, VIC, Australia",Bulleen,-37.772684,145.092133,f,Heidelberg Railway Station (Heidelberg)
1,10803,2.020000e+13,7/12/2018,Room in Cool Deco Apartment in Brunswick,38901,"Brunswick East, VIC, Australia",Brunswick East,-37.766505,144.980736,t,Northcote Railway Station (Northcote)
2,12936,2.020000e+13,7/12/2018,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,50121,"St Kilda, VIC, Australia",St Kilda,-37.859755,144.977369,t,Windsor Railway Station (Windsor)
3,15246,2.020000e+13,7/12/2018,Large private room-close to city,59786,"Thornbury, VIC, Australia",Thornbury,-37.758971,144.989228,t,Croxton Railway Station (Northcote)
4,16760,2.020000e+13,7/12/2018,Melbourne BnB near City & Sports,65090,"St Kilda East, VIC, Australia",St Kilda East,-37.864530,144.992238,t,Balaclava Railway Station (Balaclava)


In [25]:
listing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22895 entries, 0 to 22894
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 22895 non-null  int64  
 1   scrape_id          22895 non-null  float64
 2   last_scraped       22895 non-null  object 
 3   name               22892 non-null  object 
 4   host_id            22895 non-null  int64  
 5   street             22895 non-null  object 
 6   city               22872 non-null  object 
 7   latitude           22895 non-null  float64
 8   longitude          22895 non-null  float64
 9   is_location_exact  22895 non-null  object 
 10  STOP_NAME          22895 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 1.9+ MB


In [26]:
# create one dataframe with the new info
listing_final_df = pd.merge(listing_df,station_df[['STOP_NAME','LATITUDE','LONGITUDE']],on='STOP_NAME', how='left')
 
listing_final_df.head()

,id,scrape_id,last_scraped,name,host_id,street,city,latitude,longitude,is_location_exact,STOP_NAME,LATITUDE,LONGITUDE
0,9835,2.020000e+13,7/12/2018,Beautiful Room & House,33057,"Bulleen, VIC, Australia",Bulleen,-37.772684,145.092133,f,Heidelberg Railway Station (Heidelberg),-37.757073,145.060686
1,10803,2.020000e+13,7/12/2018,Room in Cool Deco Apartment in Brunswick,38901,"Brunswick East, VIC, Australia",Brunswick East,-37.766505,144.980736,t,Northcote Railway Station (Northcote),-37.769865,144.995277
2,12936,2.020000e+13,7/12/2018,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,50121,"St Kilda, VIC, Australia",St Kilda,-37.859755,144.977369,t,Windsor Railway Station (Windsor),-37.856053,144.992035
3,15246,2.020000e+13,7/12/2018,Large private room-close to city,59786,"Thornbury, VIC, Australia",Thornbury,-37.758971,144.989228,t,Croxton Railway Station (Northcote),-37.764101,144.997056
4,16760,2.020000e+13,7/12/2018,Melbourne BnB near City & Sports,65090,"St Kilda East, VIC, Australia",St Kilda East,-37.864530,144.992238,t,Balaclava Railway Station (Balaclava),-37.869488,144.993512


In [27]:
#    Calculate the great circle distance between two points 
#    on the earth (specified in decimal degrees)

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [29]:
# Creating a new column to generate the output by passing lat long information to Haversine Equation
listing_final_df['station_distance'] = [haversine(listing_final_df.longitude[i],listing_final_df.latitude[i],listing_final_df.LONGITUDE[i],listing_final_df.LATITUDE[i]) for i in range(len(listing_final_df))]
listing_final_df['station_distance'] = listing_final_df['station_distance'].round(decimals=1)
# Printing the data table 
listing_final_df.head()


,id,scrape_id,last_scraped,name,host_id,street,city,latitude,longitude,is_location_exact,STOP_NAME,LATITUDE,LONGITUDE,station_distance
0,9835,2.020000e+13,7/12/2018,Beautiful Room & House,33057,"Bulleen, VIC, Australia",Bulleen,-37.772684,145.092133,f,Heidelberg Railway Station (Heidelberg),-37.757073,145.060686,3.3
1,10803,2.020000e+13,7/12/2018,Room in Cool Deco Apartment in Brunswick,38901,"Brunswick East, VIC, Australia",Brunswick East,-37.766505,144.980736,t,Northcote Railway Station (Northcote),-37.769865,144.995277,1.3
2,12936,2.020000e+13,7/12/2018,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,50121,"St Kilda, VIC, Australia",St Kilda,-37.859755,144.977369,t,Windsor Railway Station (Windsor),-37.856053,144.992035,1.4
3,15246,2.020000e+13,7/12/2018,Large private room-close to city,59786,"Thornbury, VIC, Australia",Thornbury,-37.758971,144.989228,t,Croxton Railway Station (Northcote),-37.764101,144.997056,0.9
4,16760,2.020000e+13,7/12/2018,Melbourne BnB near City & Sports,65090,"St Kilda East, VIC, Australia",St Kilda East,-37.864530,144.992238,t,Balaclava Railway Station (Balaclava),-37.869488,144.993512,0.6


In [34]:
listing_final_df[['id', 'latitude', 'longitude', 'STOP_NAME', 'LATITUDE', 'LONGITUDE', 'station_distance']].to_csv('listing_station_coords.csv')